In [ ]:
from rdflib import Graph, RDF, RDFS, Namespace, Literal, URIRef 

G is the graph to which you add the triples. This graph will be transformed into a turtle file later. 
KIN is the shortened version of the namespace. And g.bind makes sure that "kinship1" is actually used in the ontology.

In [ ]:
g = Graph()

# KIN = Namespace('http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#')
KIN = Namespace('http://www.semanticweb.org/combots#')

g.bind('kinship1',KIN)

Here are some necessary functions:
* serialize shows the graph in a readable format.
* load is used to import the ontology.
* save is used to save the ontology.
    

In [ ]:
def serialize():
    print(g.serialize(format='turtle').decode("utf-8"))

        
def load(filename):
    with open(filename, 'r') as f:
        g.load(f, format='turtle') 
    
def save(filename):
    with open(filename, 'wb') as f:
        g.serialize(f, format='turtle') 

I created an empty ontology that is imported here. (When you add anything to g after this and then run this code block again, it will remember all the added relations.)

In [ ]:
load('Ontologies/empty_ontology.ttl')
serialize()

This is how you add triples to the graph. 

In [ ]:
g.add((KIN.Monica, KIN.has_Brother, KIN.Ross))
g.add((KIN.Ross, KIN.has_Sister, KIN.Monica))
g.add((KIN.Ross, KIN.has_Father, KIN.Jack))
g.add((KIN.Ross, RDFS.domain, KIN.man))

Then you can use serialize() to see if all the relations are actually added. So for the example, Monica has a brother named Ross. And Ross has a sister Monica, a father Jack and he is a man.

In [ ]:
serialize()

Here you can save your graph to the turle file.

In [ ]:
save('empty_ontology.ttl')

After running this code the ontology will not be empty anymore. So you can clear everything by using remove() 

In [ ]:
g.remove((KIN.Monica, KIN.has_Brother, KIN.Ross))
g.remove((KIN.Ross, KIN.has_Sister, KIN.Monica))
g.remove((KIN.Ross, KIN.has_Father, KIN.Jack))
g.remove((KIN.Ross, RDFS.domain, KIN.man))
serialize()

And then you have to save again ofcourse

In [ ]:
save('empty_ontology.ttl')

# Your implementation here

## Importing our triples

In [1]:
import pickle
with open("./gold_relations.pkl", "rb") as infile:
    gold_relations = pickle.load(infile)
with open("./data/predictions_combined_rules.pkl", "rb") as infile:
    predicted_relations = pickle.load(infile)

## Converting strings to match ontology/reasoner format

In [ ]:
def convert_relation(string):
    '''
    Takes a string representing a relation as output by the kinship detecting system
    and converts it to the string required by the reasoner.
    '''
    relation_name = string.split("-")[1]
    
    if relation_name == "daughter":
        new_relation = "Dauter"
    elif relation_name == "parent":
        new_relation = "parent"
    elif relation_name == "twin":
        new_relation = "twin"
    else:
        new_relation = relation_name.capitalize()
    
    return "has_"+new_relation
    

In [ ]:
def convert_name(string):
    '''
    takes a string of a character name as output by the kinship detecting system
    and converts it to a string usable by the ontology and reasoner (no spaces)
    '''
    name_split = string.split(" ")
    new_name = "_".join(name_split)
    return new_name

## Writing raw output file
The ontology created here includes all of the raw output from the kinship detection script

In [ ]:
# convert predicted_relations to the new format
list_of_triples = []
for triple in predicted_relations:
    new_triple = [convert_name(triple[0]), convert_relation(triple[1]), convert_name(triple[2])]
    list_of_triples.append(new_triple)

In [ ]:
# populate ontology with list_of_triples
for triple in list_of_triples:
    base = 'http://www.semanticweb.org/combots#'
    new_triple = tuple(URIRef(base+i) for i in triple)
    g.add(new_triple)

In [ ]:
serialize()

In [ ]:
save('./Ontologies/raw_output_ontology.ttl')

## Writing correct output file
The ontology created here includes only those relations from our output that also exist in the gold relations

In [ ]:
list_of_triples = []
for triple in predicted_relations:
    if triple in gold_relations:
        new_triple = [convert_name(triple[0]), convert_relation(triple[1]), convert_name(triple[2])]
        list_of_triples.append(new_triple)
list_of_triples

In [ ]:
# populate ontology with list_of_triples
for triple in list_of_triples:
    base = 'http://www.semanticweb.org/combots#'
    new_triple = tuple(URIRef(base+i) for i in triple)
    g.add(new_triple)

In [ ]:
save('./Ontologies/correct_output_ontology.ttl')

### Some things to keep in mind while creating the turtle file

Below are the properties that I use, so please use the same, because otherwise the reasoner does not work. The properties are defined in the URI after the #. So for the first one "child_property" the relation that you should use is "has_Child".

child_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Child

parent_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_parent

grandchild_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Grandchild

grandparent_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Grandparent

husband_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Husband

wife_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Wife

ex_husband_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Ex_husband

ex_wife_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Ex_wife

cousin_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Cousin

partner_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Partner

father_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Father

mother_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Mother

son_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Son

daugther_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Dauther

grandson_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Grandson

granddaugther_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Granddaughter

grandfather_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Grandfather

grandmother_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Grandmother

brother_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Brother

sister_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Sister

twin_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_twin

sibling_property http://www.semanticweb.org/selin/ontologies/2020/10/kinship1#has_Sibling

I know you are not adding any gender right know. But for the reasoner to work more efficiently it would be nice to manually add some genders. Because that ensures more inference(exept if you have a lot of relations that depend on gender(like brother), because then the reasoner should be able to add genders itself. 